## Inbäddningar

I vårt tidigare exempel arbetade vi med högdimensionella bag-of-words-vektorer med längden `vocab_size`, och vi konverterade uttryckligen lågdimensionella positionsrepresentationsvektorer till glesa one-hot-representationer. Denna one-hot-representation är inte minneseffektiv. Dessutom behandlas varje ord oberoende av varandra, så one-hot-kodade vektorer uttrycker inte semantiska likheter mellan ord.

I denna enhet kommer vi att fortsätta utforska **News AG**-datasetet. För att börja, låt oss ladda data och hämta några definitioner från föregående enhet.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Vad är en embedding?

Idén med **embedding** är att representera ord med lågdimensionella täta vektorer som reflekterar ordets semantiska betydelse. Vi kommer senare att diskutera hur man bygger meningsfulla ordembeddings, men för tillfället kan vi tänka på embeddings som ett sätt att minska dimensionaliteten hos en ordvektor.

En embedding-lager tar alltså ett ord som input och producerar en utgångsvektor med en specificerad `embedding_size`. På ett sätt liknar det ett `Dense`-lager, men istället för att ta en one-hot-kodad vektor som input kan det ta ett ordnummer.

Genom att använda ett embedding-lager som det första lagret i vårt nätverk kan vi byta från bag-of-words till en **embedding bag**-modell, där vi först konverterar varje ord i vår text till motsvarande embedding och sedan beräknar någon aggregeringsfunktion över alla dessa embeddings, såsom `sum`, `average` eller `max`.

![Bild som visar en embedding-klassificerare för fem sekvensord.](../../../../../translated_images/sv/embedding-classifier-example.b77f021a7ee67eee.webp)

Vårt klassificeringsnätverk består av följande lager:

* `TextVectorization`-lager, som tar en sträng som input och producerar en tensor av tokennummer. Vi kommer att specificera en rimlig vokabulärstorlek `vocab_size` och ignorera mindre frekvent använda ord. Input-formen kommer att vara 1, och output-formen kommer att vara $n$, eftersom vi får $n$ tokens som resultat, där varje token innehåller nummer från 0 till `vocab_size`.
* `Embedding`-lager, som tar $n$ nummer och reducerar varje nummer till en tät vektor med en given längd (100 i vårt exempel). Således kommer input-tensorn med formen $n$ att transformeras till en $n\times 100$-tensor.
* Aggregeringslager, som tar medelvärdet av denna tensor längs den första axeln, dvs. det kommer att beräkna medelvärdet av alla $n$ input-tensorer som motsvarar olika ord. För att implementera detta lager kommer vi att använda ett `Lambda`-lager och skicka in funktionen för att beräkna medelvärdet. Output kommer att ha formen 100 och kommer att vara den numeriska representationen av hela input-sekvensen.
* Slutligt `Dense` linjärt klassificeringslager.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


I sammanfattningen, i kolumnen **output shape**, motsvarar den första tensordimensionen `None` minibatch-storleken, och den andra motsvarar längden på token-sekvensen. Alla token-sekvenser i minibatchen har olika längder. Vi kommer att diskutera hur man hanterar detta i nästa avsnitt.

Nu tränar vi nätverket:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Observera** att vi bygger en vektorisering baserad på en delmängd av data. Detta görs för att påskynda processen, och det kan resultera i en situation där inte alla token från vår text finns med i vokabulären. I sådana fall kommer dessa token att ignoreras, vilket kan leda till något lägre noggrannhet. Dock ger en delmängd av text ofta en bra uppskattning av vokabulären i verkliga livet.


### Hantera variabla sekvensstorlekar

Låt oss förstå hur träning sker i minibatcher. I exemplet ovan har indatatensorn dimensionen 1, och vi använder minibatcher med längden 128, så den faktiska storleken på tensorn är $128 \times 1$. Antalet token i varje mening är dock olika. Om vi applicerar `TextVectorization`-lagret på ett enda indata, är antalet returnerade token olika beroende på hur texten tokeniseras:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Men när vi använder vektoriseringen på flera sekvenser måste den producera en tensor med rektangulär form, så den fyller oanvända element med PAD-token (vilket i vårt fall är noll):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Här kan vi se inbäddningarna:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Observera**: För att minimera mängden utfyllnad kan det i vissa fall vara vettigt att sortera alla sekvenser i datasetet i stigande ordning efter längd (eller, mer exakt, antalet token). Detta säkerställer att varje minibatch innehåller sekvenser med liknande längd.


## Semantiska inbäddningar: Word2Vec

I vårt tidigare exempel lärde sig inbäddningslagret att kartlägga ord till vektorrepresentationer, men dessa representationer hade ingen semantisk betydelse. Det vore bra att lära sig en vektorrepresentation där liknande ord eller synonymer motsvarar vektorer som ligger nära varandra baserat på någon vektordistans (till exempel euklidisk distans).

För att göra detta behöver vi förträna vår inbäddningsmodell på en stor samling text med hjälp av en teknik som [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Den bygger på två huvudsakliga arkitekturer som används för att skapa en distribuerad representation av ord:

 - **Continuous bag-of-words** (CBoW), där vi tränar modellen att förutsäga ett ord utifrån den omgivande kontexten. Givet ngrammet $(W_{-2},W_{-1},W_0,W_1,W_2)$ är målet för modellen att förutsäga $W_0$ utifrån $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** är motsatsen till CBoW. Modellen använder det omgivande fönstret av kontextord för att förutsäga det aktuella ordet.

CBoW är snabbare, medan skip-gram är långsammare men gör ett bättre jobb med att representera sällsynta ord.

![Bild som visar både CBoW- och Skip-Gram-algoritmer för att konvertera ord till vektorer.](../../../../../translated_images/sv/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

För att experimentera med Word2Vec-inbäddningen förtränad på Google News-datasetet kan vi använda **gensim**-biblioteket. Nedan hittar vi de ord som är mest liknande 'neural'.

> **Note:** När du först skapar ordvektorer kan nedladdningen ta lite tid!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Vi kan också extrahera vektorinbäddningen från ordet, för att användas vid träning av klassificeringsmodellen. Inbäddningen har 300 komponenter, men här visar vi endast de första 20 komponenterna av vektorn för tydlighet:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Det fantastiska med semantiska inbäddningar är att du kan manipulera vektorkodningen baserat på semantik. Till exempel kan vi be om att hitta ett ord vars vektorrepresentation är så nära som möjligt orden *kung* och *kvinna*, och så långt som möjligt från ordet *man*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Ett exempel ovan använder lite intern GenSym-magi, men den underliggande logiken är faktiskt ganska enkel. En intressant sak med inbäddningar är att du kan utföra normala vektoroperationer på inbäddningsvektorer, och det skulle reflektera operationer på ords **betydelser**. Exemplet ovan kan uttryckas i termer av vektoroperationer: vi beräknar vektorn som motsvarar **KING-MAN+WOMAN** (operationerna `+` och `-` utförs på vektorrepresentationer av motsvarande ord), och sedan hittar vi det närmaste ordet i ordboken till den vektorn:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Vi var tvungna att lägga till små koefficienter till *man*- och *kvinna*-vektorerna - prova att ta bort dem för att se vad som händer.

För att hitta den närmaste vektorn använder vi TensorFlow-verktyg för att beräkna en vektor av avstånd mellan vår vektor och alla vektorer i vokabulären, och sedan hittar vi indexet för det minsta ordet med hjälp av `argmin`.


Medan Word2Vec verkar vara ett utmärkt sätt att uttrycka ordsemantik, har det många nackdelar, inklusive följande:

* Både CBoW- och skip-gram-modeller är **prediktiva inbäddningar**, och de tar endast hänsyn till lokal kontext. Word2Vec utnyttjar inte global kontext.
* Word2Vec tar inte hänsyn till ords **morfologi**, det vill säga det faktum att ordets betydelse kan bero på olika delar av ordet, såsom roten.

**FastText** försöker övervinna den andra begränsningen och bygger vidare på Word2Vec genom att lära sig vektorrepresentationer för varje ord och de teckenn-gram som finns inom varje ord. Värdena för representationerna genomsnittas sedan till en vektor vid varje träningssteg. Även om detta lägger till mycket extra beräkning under förträningen, möjliggör det att ordinbäddningar kan koda information på subordnivå.

En annan metod, **GloVe**, använder en annan strategi för ordinbäddningar, baserad på faktorisering av ord-kontext-matrisen. Först bygger den en stor matris som räknar antalet förekomster av ord i olika kontexter, och sedan försöker den representera denna matris i lägre dimensioner på ett sätt som minimerar rekonstruktionsförlusten.

Biblioteket gensim stöder dessa ordinbäddningar, och du kan experimentera med dem genom att ändra modellens laddningskod ovan.


## Använda förtränade inbäddningar i Keras

Vi kan modifiera exemplet ovan för att förfylla matrisen i vårt inbäddningslager med semantiska inbäddningar, såsom Word2Vec. Ordförråden för den förtränade inbäddningen och textkorpusen kommer sannolikt inte att matcha, så vi måste välja ett. Här utforskar vi de två möjliga alternativen: att använda tokenizer-ordförrådet och att använda ordförrådet från Word2Vec-inbäddningar.

### Använda tokenizer-ordförrådet

När vi använder tokenizer-ordförrådet kommer vissa av orden i ordförrådet att ha motsvarande Word2Vec-inbäddningar, medan andra kommer att saknas. Givet att vår ordförrådsstorlek är `vocab_size` och att Word2Vec-inbäddningsvektorns längd är `embed_size`, kommer inbäddningslagret att representeras av en viktmatris med formen `vocab_size`$\times$`embed_size`. Vi kommer att fylla denna matris genom att gå igenom ordförrådet:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


För ord som inte finns i Word2Vec-ordförrådet kan vi antingen lämna dem som nollor eller generera en slumpmässig vektor.

Nu kan vi definiera ett inbäddningslager med förtränade vikter:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Observera**: Notera att vi sätter `trainable=False` när vi skapar `Embedding`, vilket innebär att vi inte tränar om Embedding-lagret. Detta kan leda till något lägre noggrannhet, men det påskyndar träningen.

### Använda inbäddningsordförråd

Ett problem med det tidigare tillvägagångssättet är att ordförråden som används i TextVectorization och Embedding är olika. För att lösa detta problem kan vi använda en av följande lösningar:
* Träna om Word2Vec-modellen på vårt ordförråd.
* Ladda vår dataset med ordförrådet från den förtränade Word2Vec-modellen. Ordförråd som används för att ladda datasetet kan specificeras under laddningen.

Det senare tillvägagångssättet verkar enklare, så låt oss implementera det. Först och främst kommer vi att skapa ett `TextVectorization`-lager med det specificerade ordförrådet, hämtat från Word2Vec-inbäddningarna:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensim-biblioteket för ordbaser innehåller en praktisk funktion, `get_keras_embeddings`, som automatiskt skapar det motsvarande Keras-embeddingslagret åt dig.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


En av anledningarna till att vi inte ser högre noggrannhet är att vissa ord från vår dataset saknas i den förtränade GloVe-ordlistan och därför i princip ignoreras. För att lösa detta kan vi träna våra egna inbäddningar baserade på vår dataset.


## Kontextuella inbäddningar

En viktig begränsning med traditionella förtränade inbäddningsrepresentationer som Word2Vec är att även om de kan fånga en viss betydelse av ett ord, kan de inte skilja mellan olika betydelser. Detta kan orsaka problem i nedströmsmodeller.

Till exempel har ordet 'play' olika betydelser i dessa två meningar:
- Jag gick på en **pjäs** på teatern.
- John vill **leka** med sina vänner.

De förtränade inbäddningarna vi pratade om representerar båda betydelserna av ordet 'play' i samma inbäddning. För att övervinna denna begränsning behöver vi bygga inbäddningar baserade på **språkmodellen**, som är tränad på en stor textkorpus och *vet* hur ord kan sättas ihop i olika kontexter. Att diskutera kontextuella inbäddningar ligger utanför ramen för denna handledning, men vi kommer tillbaka till dem när vi pratar om språkmodeller i nästa enhet.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
